# Areas of Interest to Django
Import SpatiaLite records to a GeoDjango database

### Import libraries, define settings, import model

In [1]:
import os
import django
from django.db import transaction
from django.contrib.gis.geos import GEOSGeometry
import sqlite3
from asgiref.sync import sync_to_async

import sys; sys.path.append('../../')
os.environ['DJANGO_SETTINGS_MODULE'] = 'gaia.settings'
django.setup()

from whale.models import EarthExplorer as DjangoTable
from whale.models import AreaOfInterest as ForeignDjangoTable

### User defined variables

In [2]:
sdb = "C:/gis/gaia/data/databases/gaia.db"
sdb_table_name = "ee"
geometry_col = 'bounds'

### Perform async insertion to database

In [5]:
@sync_to_async
def insert_data(**kwargs):
    """ Insert Areas of Interest into the GeoDjango database"""
    if 'geometry_text' in kwargs:
        kwargs['geometry'] = GEOSGeometry(kwargs.pop('geometry_text'))
    if 'aoi_id' in kwargs:
        kwargs['aoi_id'] = ForeignDjangoTable.objects.get(id=kwargs.pop('aoi_id'))
    
    DjangoTable.objects.create(**kwargs)

conn = sqlite3.connect(sdb)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

pragma_statement = f"PRAGMA table_info({sdb_table_name});"
print(pragma_statement)
cursor = conn.execute(pragma_statement)
columns = [row[1] for row in cursor.fetchall()]
print(columns)

columns_with_geom = [
    f"AsText({geometry_col}) AS geometry_text" if col == geometry_col else col
    for col in columns
]
print(columns_with_geom)
query = f"SELECT {', '.join(columns_with_geom)} FROM {sdb_table_name};"
print(query)
cursor.execute(query)
rows = cursor.fetchall()

conn.close()

import asyncio

async def commit_rows():
    for row in rows:
        data = dict(zip(columns, row))
        await insert_data(**data)

await commit_rows()

PRAGMA table_info(ee);
['entity_id', 'aoi_id', 'catalog_id', 'acquisition_date', 'vendor', 'vendor_id', 'cloud_cover', 'satellite', 'sensor', 'number_of_bands', 'map_projection', 'datum', 'processing_level', 'file_format', 'license_id', 'sun_azimuth', 'sun_elevation', 'pixel_size_x', 'pixel_size_y', 'license_uplift_update', 'event', 'event_date', 'date_entered', 'center_latitude_dec', 'center_longitude_dec', 'thumbnail', 'publish_date', 'bounds']
['entity_id', 'aoi_id', 'catalog_id', 'acquisition_date', 'vendor', 'vendor_id', 'cloud_cover', 'satellite', 'sensor', 'number_of_bands', 'map_projection', 'datum', 'processing_level', 'file_format', 'license_id', 'sun_azimuth', 'sun_elevation', 'pixel_size_x', 'pixel_size_y', 'license_uplift_update', 'event', 'event_date', 'date_entered', 'center_latitude_dec', 'center_longitude_dec', 'thumbnail', 'publish_date', 'AsText(bounds) AS geometry_text']
SELECT entity_id, aoi_id, catalog_id, acquisition_date, vendor, vendor_id, cloud_cover, satellit

# End